# 4 - Convolutional Sentiment Analysis

In the previous notebooks, we managed to achieve a test accuracy of ~85% using RNNs and an implementation of the [Bag of Tricks for Efficient Text Classification](https://arxiv.org/abs/1607.01759) model. In this final notebook, we will be using a *convolutional neural network* (CNN) to conduct sentiment analysis, implementing the model from [Convolutional Neural Networks for Sentence Classification](https://arxiv.org/abs/1408.5882).

**Note**: I am not aiming to give a comprehensive introduction and explanation of CNNs. For a better and more in-depth explanation check out [here](https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/) and [here](https://cs231n.github.io/convolutional-networks/).

Traditionally, CNNs are used to analyse images and are made up of one or more *convolutional* layers, followed by one or more linear layers. The convolutional layers use filters (also called *kernels* or *receptive fields*) which scan across an image and produce a processed version of the image. This processed version of the image can be fed into another convolutional layer or a linear layer. Each filter has a shape, e.g. a 3x3 filter covers a 3 pixel wide and 3 pixel high area of the image, and each element of the filter has a weight associated with it, the 3x3 filter would have 9 weights. In traditional image processing these weights were specified by hand by engineers, however the main advantage of the convolutional layers in neural networks is that these weights are learned via backpropagation. 

The intuitive idea behind learning the weights is that your convolutional layers act like *feature extractors*, extracting parts of the image that are most important for your CNN's goal, e.g. if using a CNN to detect faces in an image, the CNN may be looking for features such as the existance of a nose, mouth or a pair of eyes in the image.

So why use CNNs on text? In the same way that a 3x3 filter can look over a patch of an image, a 1x2 filter can look over a 2 sequential words in a piece of text, i.e. a bi-gram. In the previous tutorial we looked at the FastText model which used bi-grams by explicitly adding them to the end of a text, in this CNN model we will instead use multiple filters of different sizes which will look at the bi-grams (a 1x2 filter), tri-grams (a 1x3 filter) and n-grams (a 1x$n$ filter) within the text.

The intuition here is that the appearance of certain bi-grams, tri-grams and n-grams within the review will be a good indication of the final sentiment.

## Preparing Data

As in the previous notebooks, we'll prepare the data. 

Unlike the previous notebook with the FastText model, we no longer explicitly need to create the bi-grams and append them to the end of the sentence.

In [1]:
import pickle
import csv
import torch
# from torchtext import data
# from torchtext import datasets
import random
import numpy as np



SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


In [2]:
with open('/Users/lyra/Downloads/0.pickle', 'rb') as f:
    train_data_pickle = pickle.load(f)

train_data_path = '/Users/lyra/Downloads/0.csv'
train_data_len = 0
per_len = 100

with open(train_data_path, 'w') as csvfile:
    fieldnames = ['label', 'text']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_MINIMAL)
    
    for row in train_data_pickle:
        for i in range(0, len(row[0]), per_len):
            flag = False
            for j in range(i, min(i+per_len, len(row[0]))):
                if row[0][j] != 0.0:
                    flag = True
                    break
            if flag == False:
                continue
            train_data_len += 2
            batch = row[0][i:i+per_len]
            batch_fake = row[1][i:i+per_len]
            writer.writerow({'label': 0.0, 'text': batch.tolist()})
            writer.writerow({'label': 1.0, 'text': batch_fake.tolist()})

In [3]:
train_data = np.zeros((train_data_len, per_len + 1), dtype = int)
k = 0
for row in train_data_pickle:
    for i in range(0, len(row[0]), per_len):
        flag = False
        for j in range(i, min(i+per_len, len(row[0]))):
            if row[0][j] != 0.0:
                flag = True
                break
        if flag == False:
            continue
        for j in range(i, min(i+per_len, len(row[0]))):
            train_data[k][j-i+1] = int(row[0][j] * 100)
            train_data[k][0] = 0
            train_data[k+1][j-i+1] = int(row[1][j] * 100)
            train_data[k+1][0] = 1
        k += 2

np.random.shuffle(train_data)

train_data_text = torch.from_numpy(train_data[:int(train_data_len*0.9), 1:])
train_data_label = train_data[:int(train_data_len*0.9), 0].astype(np.float32)
train_data_label = torch.from_numpy(train_data_label)

valid_data_text = torch.from_numpy(train_data[int(train_data_len*0.9):, 1:])
valid_data_label = train_data[int(train_data_len*0.9):, 0].astype(np.float32)
valid_data_label = torch.from_numpy(valid_data_label)

In [4]:
with open('/Users/lyra/Downloads/1.pickle', 'rb') as f:
    test_data_pickle = pickle.load(f)

test_data_len = 0

for row in test_data_pickle:
    for i in range(0, len(row[0]), per_len):
        flag = False
        for j in range(i, min(i+per_len, len(row[0]))):
            if row[0][j] != 0.0:
                flag = True
                break
        if flag == False:
            continue
        test_data_len += 2
        
test_data = np.zeros((test_data_len, per_len + 1), dtype = int)
k = 0
for row in test_data_pickle:
    for i in range(0, len(row[0]), per_len):
        flag = False
        for j in range(i, min(i+per_len, len(row[0]))):
            if row[0][j] != 0.0:
                flag = True
                break
        if flag == False:
            continue
        for j in range(i, min(i+per_len, len(row[0]))):
            test_data[k][j-i+1] = int(row[0][j] * 100)
            test_data[k][0] = 0
            test_data[k+1][j-i+1] = int(row[1][j] * 100)
            test_data[k+1][0] = 1
        k += 2

np.random.shuffle(test_data)

test_data_text = torch.from_numpy(test_data[:, 1:])
test_data_label = test_data[:, 0].astype(np.float32)
test_data_label = torch.from_numpy(test_data_label)

As before, we create the iterators.

In [5]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
#     (train_data, valid_data, test_data), 
#     batch_size=BATCH_SIZE, 
#     device=device)

In [11]:
print(train_data.shape)
train_data_len = train_data_label.shape[0]
print(train_data_len)
print(train_data_label[0])
print(train_data_text[0])
valid_data_len = valid_data_label.shape[0]
print(valid_data_len)
print(valid_data_label[0])
print(valid_data_text[0])
print(test_data_len, test_data.shape)
print(test_data_label[0])
print(test_data_text[0])

(20326, 101)
18293
tensor(1.)
tensor([    0,     0,     0,     1,     5,    16,    42,    99,   213,   419,
          766,  1303,  2071,  3086,  4324,  5718, 15863, 25903, 35714, 35816,
        35772, 27276, 19151, 19551, 27807, 34692, 24584, 14422,  5480,  4574,
         3979,  3783,  4018,  4652, 11175, 18735, 26561, 28331, 29082, 28910,
        29209, 29676, 22854, 15500,  7681,  7527,  7667, 16903, 24615, 31439,
        29019, 27790, 27864, 27946, 29607, 31184, 32308, 31979, 29461, 26127,
        22815, 21049, 20615, 20245, 20016, 20017, 16235, 12503, 13127, 17795,
        22632, 23051, 23266, 23469, 23957, 24753, 25598, 26228, 26625, 26811,
        26873, 26815, 26675, 26591, 26633, 26671, 26545, 27371, 28634, 28883,
        27566, 25812, 25239, 25180, 25130, 24980, 24761, 24489, 24107, 24396])
2033
tensor(1.)
tensor([35200, 36042, 36515, 36693, 36662, 36573, 36505, 36473, 37373, 38150,
        38038, 37276, 36815, 37304, 37556, 37556, 37427, 37298, 37907, 38517,
        38332, 37

## Build the Model

Now to build our model.

The first major hurdle is visualizing how CNNs are used for text. Images are typically 2 dimensional (we'll ignore the fact that there is a third "colour" dimension for now) whereas text is 1 dimensional. However, we know that the first step in almost all of our previous tutorials (and pretty much all NLP pipelines) is converting the words into word embeddings. This is how we can visualize our words in 2 dimensions, each word along one axis and the elements of vectors aross the other dimension. Consider the 2 dimensional representation of the embedded sentence below:

![](https://i.imgur.com/ci1h9hv.png)

We can then use a filter that is **[n x emb_dim]**. This will cover $n$ sequential words entirely, as their width will be `emb_dim` dimensions. Consider the image below, with our word vectors are represented in green. Here we have 4 words with 5 dimensional embeddings, creating a [4x5] "image" tensor. A filter that covers two words at a time (i.e. bi-grams) will be **[2x5]** filter, shown in yellow, and each element of the filter with have a _weight_ associated with it. The output of this filter (shown in red) will be a single real number that is the weighted sum of all elements covered by the filter.

![](https://i.imgur.com/QlXduXu.png)

The filter then moves "down" the image (or across the sentence) to cover the next bi-gram and another output (weighted sum) is calculated. 

![](https://i.imgur.com/wuA330x.png)

Finally, the filter moves down again and the final output for this filter is calculated.

![](https://i.imgur.com/gi1GaEz.png)

In our case (and in the general case where the width of the filter equals the width of the "image"), our output will be a vector with number of elements equal to the height of the image (or lenth of the word) minus the height of the filter plus one, $4-2+1=3$ in this case.

This example showed how to calculate the output of one filter. Our model (and pretty much all CNNs) will have lots of these filters. The idea is that each filter will learn a different feature to extract. In the scenario of analysing text, we are hoping each of the **[2 x emb_dim]** filters will be looking for the occurence of different bi-grams. 

In our model, we will also have different sizes of filters, heights of 3, 4 and 5, with 100 of each of them. The intuition is that we will be looking for the occurence of different tri-grams, 4-grams and 5-grams that are relevant for analysing sentiment of movie reviews.

The next step in our model is to use *pooling* (specifically *max pooling*) on the output of the convolutional layers. This is similar to the FastText model where we performed the average over each of the word vectors, implemented by the `F.avg_pool2d` function, however instead of taking the average over a dimension, we are taking the maximum value over a dimension. Below an example of taking the maximum value (0.9) from the output of the convolutional layer on the example sentence (not shown in the activation function applied to the output of the convolutions).

![](https://i.imgur.com/gzkS3ze.png)

The idea here is that the maximum value is the "most important" feature for determining the sentiment of the review, which corresponds to the "most important" n-gram within the review. How do we know what the "most important" n-gram is? Luckily, we don't have to! Through backpropagation, the weights of the filters are changed so that whenever certain n-grams that are highly indicative of the sentiment are seen, the output of the filter is a "high" value. This "high" value then passes through the max pooling layer if it is the maximum value in the output. 

As our model has 100 filters of 3 different sizes, that means we have 300 different n-grams the model thinks are important. We concatenate these together into a single vector and pass them through a linear layer to predict the sentiment. We can think of the weights of this linear layer as "weighting up the evidence" from each of the 300 n-grams and making a final decision. 

### Implementation Details

We implement the convolutional layers with `nn.Conv2d`. The `in_channels` argument is the number of "channels" in your image going into the convolutional layer. In actual images this is usually 3 (one channel for each of the red, blue and green channels), however when using text we only have a single channel, the text itself. The `out_channels` is the number of filters and the `kernel_size` is the size of the filters. Each of our `kernel_size`s is going to be **[n x emb_dim]** where $n$ is the size of the n-grams.

In PyTorch, RNNs want the input with the batch dimension second, whereas CNNs want the batch dimension first. Thus, the first thing we do to our input is `permute` it to make it the correct shape. We then pass the sentence through an embedding layer to get our embeddings. The second dimension of the input into a `nn.Conv2d` layer must be the channel dimension. As text technically does not have a channel dimension, we `unsqueeze` our tensor to create one. This matches with our `in_channels=1` in the initialization of our convolutional layers. 

We then pass the tensors through the convolutional and pooling layers, using the `ReLU` activation function after the convolutional layers. Another nice feature of the pooling layers is that they handle sentences of different lengths. The size of the output of the convolutional layer is dependent on the size of the input to it, and different batches contain sentences of different lengths. Without the max pooling layer the input to our linear layer would depend on the size of the input sentence (not what we want). One option to rectify this would be to trim/pad all sentences to the same length, however with the max pooling layer we always know the input to the linear layer will be the total number of filters. **Note**: there an exception to this if your sentence(s) are shorter than the largest filter used. You will then have to pad your sentences to the length of the largest filter. In the IMDb data there are no reviews only 5 words long so we don't have to worry about that, but you will if you are using your own data.

Finally, we perform dropout on the concatenated filter outputs and then pass them through a linear layer to make our predictions.

In [12]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0],embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1],embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2],embedding_dim))
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        #x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

Currently the `CNN` model can only use 3 different sized filters, but we can actually improve the code of our model to make it more generic and take any number of filters.

We do this by placing all of our convolutional layers in a  `nn.ModuleList`, a function used to hold a list of PyTorch `nn.Module`s. If we simply used a standard Python list, the modules within the list cannot be "seen" by any modules outside the list which will cause us some errors.

We can now pass an arbitrary sized list of filter sizes and the list comprehension will create a convolutional layer for each of them. Then, in the `forward` method we iterate through the list applying each convolutional layer to get a list of convolutional outputs, which we also feed through the max pooling in a list comprehension before concatenating together and passing through the dropout and linear layers.

In [13]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(fs,embedding_dim)) for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [sent len, batch size]
        
        #x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        
        #print(x.shape)
        
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

We create an instance of our `CNN` class.

In [14]:
INPUT_DIM = 100000
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5

print(INPUT_DIM, EMBEDDING_DIM)

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

100000 100


And load the pre-trained embeddings

## Train the Model

Training is the same as before. We initialize the optimizer, loss function (criterion) and place the model and criterion on the GPU (if available)

In [15]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

We implement the function to calculate accuracy...

In [16]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

We define a function for training our model...

**Note**: as we are using dropout again, we must remember to use `model.train()` to ensure the dropout is "turned on" while training.

In [17]:
def train(model, train_data_len, train_data_text, train_data_label, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch_id in range(int(train_data_len / BATCH_SIZE)):
        
        batch_text = train_data_text[batch_id * BATCH_SIZE : min((batch_id+1)*BATCH_SIZE, train_data_len), :]
        batch_label = train_data_label[batch_id * BATCH_SIZE : min((batch_id+1)*BATCH_SIZE, train_data_len)]
        
        #print(batch_text)
        #print(batch_label)
        
        optimizer.zero_grad()
        
        predictions = model(batch_text).squeeze(1)
        
        loss = criterion(predictions, batch_label)
        
        acc = binary_accuracy(predictions, batch_label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / int(train_data_len / BATCH_SIZE), epoch_acc / int(train_data_len / BATCH_SIZE)

We define a function for testing our model...

**Note**: again, as we are now using dropout, we must remember to use `model.eval()` to ensure the dropout is "turned off" while evaluating.

In [18]:
def evaluate(model, valid_data_len, valid_data_text, valid_data_label, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch_id in range(int(valid_data_len / BATCH_SIZE)):
            batch_text = valid_data_text[batch_id * BATCH_SIZE : min((batch_id+1)*BATCH_SIZE, valid_data_len), :]
            batch_label = valid_data_label[batch_id * BATCH_SIZE : min((batch_id+1)*BATCH_SIZE, valid_data_len)]

            predictions = model(batch_text).squeeze(1)
            
            loss = criterion(predictions, batch_label)
            
            acc = binary_accuracy(predictions, batch_label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / int(valid_data_len / BATCH_SIZE), epoch_acc / int(valid_data_len / BATCH_SIZE)

Finally, we train our model...

In [20]:
N_EPOCHS = 1

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_data_len, train_data_text, train_data_label, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_data_len, valid_data_text, valid_data_label, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

| Epoch: 01 | Train Loss: 0.135 | Train Acc: 95.94% | Val. Loss: 0.084 | Val. Acc: 97.53% |


...and get our best test accuracy yet! 

In [22]:
test_loss, test_acc = evaluate(model, test_data_len, test_data_text, test_data_label, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

| Test Loss: 0.055 | Test Acc: 98.14% |
